## Group 4 - Lab 6 - Python

Katiuska Olivera Quevedo (20172533) <br/> 
Rosemery Fernandez Sanchez (20172635) <br/> 
Aliro Cabrera Florez (20152034) <br/> 
Jose Uscamayta Quispe (20195674) <br/> 

In [1]:
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy


In [2]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [3]:
import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

In [4]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

### DATA 

In [5]:
growth = pyreadr.read_r("C:/Users/ok235/OneDrive/Documentos/GitHub/ECO224/Labs/data/GrowthData.RData") 


In [6]:
data1 = growth[ 'GrowthData' ]
list(data1)


['Outcome',
 'intercept',
 'gdpsh465',
 'bmp1l',
 'freeop',
 'freetar',
 'h65',
 'hm65',
 'hf65',
 'p65',
 'pm65',
 'pf65',
 's65',
 'sm65',
 'sf65',
 'fert65',
 'mort65',
 'lifee065',
 'gpop1',
 'fert1',
 'mort1',
 'invsh41',
 'geetot1',
 'geerec1',
 'gde1',
 'govwb1',
 'govsh41',
 'gvxdxe41',
 'high65',
 'highm65',
 'highf65',
 'highc65',
 'highcm65',
 'highcf65',
 'human65',
 'humanm65',
 'humanf65',
 'hyr65',
 'hyrm65',
 'hyrf65',
 'no65',
 'nom65',
 'nof65',
 'pinstab1',
 'pop65',
 'worker65',
 'pop1565',
 'pop6565',
 'sec65',
 'secm65',
 'secf65',
 'secc65',
 'seccm65',
 'seccf65',
 'syr65',
 'syrm65',
 'syrf65',
 'teapri65',
 'teasec65',
 'ex1',
 'im1',
 'xr65',
 'tot1']

In [7]:
data1.shape

(90, 63)

### We create the main variables

y = outcome: growth rate <br/>
d = treatment: initial wealth <br/>
x = controls: country characteristics <br/>

In [8]:
y = data1['Outcome']
d = data1['gdpsh465']
x = data1.drop(['Outcome','gdpsh465','intercept'], 1)

### 1. OLS without including the country characteristics

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [27]:
model = "y ~ d"
reg_ols = smf.ols(model , data=data1).fit()
print(reg_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                   0.04656
Date:                Sat, 27 Nov 2021   Prob (F-statistic):              0.830
Time:                        12:45:56   Log-Likelihood:                 140.10
No. Observations:                  90   AIC:                            -276.2
Df Residuals:                      88   BIC:                            -271.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0352      0.047      0.744      0.4

In [38]:
coef1 = reg_ols.summary2().tables[1]['Coef.']['d']
err1 =reg_ols.summary2().tables[1]['Std.Err.']['d']

The point estimate is $0.0013$ with the confidence interval ranging from -0.059 to 0.129. This suggests that increases in the country outcome are very low related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes up by 0.0013%, without controlling for countries' characteristics.

### 2. OLS including the country characteristics

Now we're gonna run an OLS regression with the controls.

In [12]:
model = "y ~ d + x"
baseline_ols = smf.ols(model , data=data1).fit()
print(baseline_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.641
Method:                 Least Squares   F-statistic:                     3.607
Date:                Sat, 27 Nov 2021   Prob (F-statistic):           0.000200
Time:                        12:41:52   Log-Likelihood:                 238.24
No. Observations:                  90   AIC:                            -352.5
Df Residuals:                      28   BIC:                            -197.5
Df Model:                          61                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2472      0.785      0.315      0.7

In [13]:
coef2= baseline_ols.summary2().tables[1]['Coef.']['d']
err2=baseline_ols.summary2().tables[1]['Std.Err.']['d']

The point estimate is $-0.0094$, with strong multicollinearity problems. This suggests that decrease in the country outcome are  related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes down by -0.009%, with control for countries' characteristics.

### Setting DML

In [14]:
def DML2_for_PLM(x, d, y, dreg, yreg, nfold):
    
    # Num ob observations
    nobs = x.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(x) ).reshape( len(x) , 1 )
    ytil = np.zeros( len(x) ).reshape( len(x) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(x))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(x[~mask,], d[~mask,])
        yfit = yreg(x[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( x[mask,] )
        yhat = yfit.predict( x[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil ), axis = 1), columns = ['ytil','dtil'])
   
    # OLS 
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit()
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result

In [15]:
y = y.to_numpy().reshape( len(y) , 1 )
d = d.to_numpy().reshape( len(y) , 1 )
x = x.to_numpy()

### 3. DML using Lasso to predict y an d

In [16]:
def dreg(x,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(x, d)
    return result

def yreg(x,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(x, y)
    return result

DML2_lasso = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8733689875934301, tolerance: 0.006185823058915516
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014199596136990648, tolerance: 2.1767558645907993e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7137783955586213, tolerance: 0.005943076296150334
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: C

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.021948330985224595, SE is equal to 0.010846452228785717


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014625138125618, tolerance: 2.2086665509966693e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8793569690354471, tolerance: 0.0066936063976616314
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014048512395354706, tolerance: 1.9393018670134024e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530:

The point estimate is $0.021$. This suggests that increases in the country outcome are very low related to the initial wealth (similar tu our first OLS) - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes up by 0.02%, with control for countries' characteristics.

### 4. DML using Post-Lasso to predict y an d.

In [17]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

In [18]:
def dreg(x,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( x , d )
    return result

def yreg( x , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( x , y )
    return result

DML2_lasso_post = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6650600503462916, tolerance: 0.006773466296298878
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6616320175598485, tolerance: 0.006773466296298878
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.015100817489352783, tolerance: 2.1954104137805862e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: C

0  
1  
2  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9604567922800377, tolerance: 0.006135364467642725
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013983293061712256, tolerance: 2.2837159026019934e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014126745759149526, tolerance: 2.2837159026019934e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:53

3  
4  
5  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8476965033970645, tolerance: 0.0065674285355812375
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010558128533904232, tolerance: 2.0912841503510302e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01128020579945515, tolerance: 2.0912841503510302e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:53

6  
7  
8  
9  
Coefficient is 0.012830760532042008, SE is equal to 0.012090856916109689


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013338332354437548, tolerance: 1.953075831421172e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8202668100997326, tolerance: 0.005951737788065438
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8516128511086607, tolerance: 0.005951737788065438
  model = cd_fast.enet_coordinate_descent(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Co

The point estimate is  0.012 . This suggests that increase in the country outcome related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes up by 0.012%.

### 5. DML using Elastic Net to predict y an d

We decided to use StandardScaler. We proved this in the commented code below. It normalizes X by subtracting the mean and dividing by the l2-norm

In [22]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [23]:
def dreg(x,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, d )
    return result

def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( x, y )
    return result

DML2_elnet = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

7  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


8  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


9  
Coefficient is -0.014428667161809263, SE is equal to 0.011134748252665343


### 6.DML using Ridge to predict y an d

In [24]:
def dreg(x,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, d )
    return result

def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, y )
    return result

DML2_ridge = DML2_for_PLM(x, d, y, dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

7  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


8  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


9  
Coefficient is -0.019883523249041614, SE is equal to 0.009980004163969549


The point estimate is -0.01988 . This suggests that decreases in the country outcome negative related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes down by -0.019%

### 7. DML using Random Forest to predict y an d.

In [25]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [29]:
x.shape

(90, 60)

In [30]:
def dreg(x,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
    return result

def yreg(x,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )
    return result

DML2_RF = DML2_for_PLM(x, d, y, dreg, yreg, 10) 

<ipython-input-30-02a4a5a85286>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-30-02a4a5a85286>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


0  


<ipython-input-30-02a4a5a85286>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-30-02a4a5a85286>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


1  


<ipython-input-30-02a4a5a85286>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-30-02a4a5a85286>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


...

7  


<ipython-input-30-02a4a5a85286>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-30-02a4a5a85286>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


8  


<ipython-input-30-02a4a5a85286>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, d )
<ipython-input-30-02a4a5a85286>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( x, y )


9  
Coefficient is -0.03939164508064591, SE is equal to 0.012209414844115212


The point estimate is -0.03939 . This suggests that decreases in the country outcome negative related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes down by -0.03%

In [31]:
mods = [DML2_lasso, DML2_lasso_post , DML2_ridge, DML2_elnet, DML2_RF]
mods_name = ["DML2_lasso", "DML2_lasso_post", 'DML2_ridge', 'DML2_elnet', 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_lasso,DML2_lasso_post,DML2_ridge,DML2_elnet,DML2_RF
RMSEY,0.000199,0.000647,1.079383e-18,2.089378e-17,0.000342
RMSED,0.021219,0.007356,4.638265e-16,1.194107e-15,0.016565


This verfies that the function DML2_for_PLM has no errors

In [32]:
np.where(DML2_lasso_post[ 'ytil' ] == 0)[0].size

0

### 8. Run the best method i.e. the best combination of methods to predict y an d

It looks like the best method for predicting Y is CV Ridge, and the best method for predicting D is CV Ridge as well

In [33]:
def dreg(x,d):
    result = standard_skl_model(  ElasticNetCV(cv = 10 , random_state = 0 , alphas = [0]) ).fit( x, d )
    return result


def yreg(x,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( x, y )
    return result

DML2_best = DML2_for_PLM(x, d, y , dreg, yreg, 10)

C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.607005466405468, tolerance: 0.006028264784721063
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd

0  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5468907631357567, tolerance: 0.006016346064420339
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = c

1  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5273895087651965, tolerance: 0.005905301055774565
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = c

...

7  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6246919549968055, tolerance: 0.005695533334304476
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = c

8  


C:\Users\ok235\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.398180348953197, tolerance: 0.005861493190376098
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\ok235\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd

9  
Coefficient is -0.004825549690434017, SE is equal to 0.006884186394445264


The point estimate is -0.004 . This suggests that increases in the country outcome are negative related to the initial wealth - if the intial wealth increase by 1% relative to a trend then the predicted outcome goes down by -0.004%

### Results 

In [39]:
table = np.zeros( ( 8 , 2 ))
table[ 0 , 0] = coef1
table[ 1 , 0] = coef2
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_ridge['coef_est']
table[ 5 , 0] = DML2_elnet['coef_est']
table[ 6 , 0] = DML2_RF['coef_est']
table[ 7 , 0] = DML2_best['coef_est']
table[ 0 , 1] = err1
table[ 1 , 1] = err2
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_ridge['se']
table[ 5 , 1] = DML2_elnet['se']
table[ 6 , 1] = DML2_RF['se']
table[ 7 , 1] = DML2_best['se']

In [40]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "OLS with controls", "Lasso", \
             "Post-Lasso","CV Elnet", "CV Ridge", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.001,0.006
OLS with controls,0.247,0.785
Lasso,0.022,0.011
Post-Lasso,0.013,0.012
CV Elnet,-0.020,0.010
CV Ridge,-0.014,0.011
Random Forest,-0.039,0.012
Best,-0.005,0.007


In [36]:
table.to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n{} &  Estimate &  Standard Error \\\\\n\\midrule\nBaseline OLS      &  0.005827 &        0.000700 \\\\\nOLS with controls &  0.247161 &        0.784502 \\\\\nLasso             &  0.021948 &        0.010846 \\\\\nPost-Lasso        &  0.012831 &        0.012091 \\\\\nCV Elnet          & -0.019884 &        0.009980 \\\\\nCV Ridge          & -0.014429 &        0.011135 \\\\\nRandom Forest     & -0.039392 &        0.012209 \\\\\nBest              & -0.004826 &        0.006884 \\\\\n\\bottomrule\n\\end{tabular}\n'